# Imports

In [409]:
#data stuff
import pandas as pd

#Modelling
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction import text
from sklearn.ensemble import RandomForestClassifier

#NLP
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import gensim

#other
from tqdm import tqdm_notebook

# Formatting X

In [170]:
files = ['@TotalTrafficMIA',
         '@TrafficAlertSFL', '@nbc6', '@CBSMiami', '@wsvn', '@MiamiHerald', '@SunSentinel', '@miaminewsnow',
         '@BeachNewsAlerts', '@WPLGLocal10', 
       '@MiamiBeachPD', '@miamibeachnews', '@FLTurnpikeSFL']

In [65]:
datum = []

for filename in tqdm_notebook(files):
    data = pd.read_csv(f'Datasets/siteloop_{filename}.csv')
    data.drop(columns=['User','User_ID','Geo'], inplace = True)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data.assign(Date=data['Date'].dt.round('H'))['Date']
    times = []
    times.append(data['Date'][0])

    for i in data.index:
        time = data['Date'][i]
        if time != times[len(times)-1]:
            times.append(time)

    dic = {'time': [], f'{filename}': []}

    for hour in times:
        total = ''
        tweets = list(data[data['Date'] == hour]['Tweet'])
        for twit in tweets:
            total += str(twit)
        dic['time'].append(hour)
        dic[f'{filename}'].append(total)
    exec(f'data_{filename[1:]} = pd.DataFrame.from_dict(dic)')
    
    exec(f'data_{filename[1:]}.set_index("time", inplace=True)')
    exec(f'data_{filename[1:]}.sort_index(inplace=True)')
    
    exec(f'datum.append(data_{filename[1:]})')
df = pd.concat(datum, axis=1)
df.fillna('', inplace=True)

In [77]:
df['ALL'] = df['@TotalTrafficMIA'] + df['@TrafficAlertSFL'] + df['@nbc6'] + df['@CBSMiami'] + df['@wsvn'] +\
            df['@MiamiHerald'] +  df['@SunSentinel'] + df['@miaminewsnow'] + df['@BeachNewsAlerts'] + df['@WPLGLocal10'] +\
            df['@MiamiBeachPD'] + df['@miamibeachnews'] + df['@FLTurnpikeSFL']

# Format X attempt 2

In [388]:
files = ['@TotalTrafficMIA', '@TrafficAlertSFL', '@nbc6', '@CBSMiami', '@wsvn', '@MiamiHerald', '@SunSentinel', 
        '@WPLGLocal10', '@miaminewsnow', '@BeachNewsAlerts', '@FLTurnpikeSFL', 
       '@MiamiBeachPD', '@miamibeachnews']
# '@breakingnewsmia',

In [389]:
datum = []

for filename in tqdm_notebook(files):
    data = pd.read_csv(f'Datasets/siteloop_{filename}.csv')
    data.drop(columns=['User','User_ID','Geo'], inplace = True)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data.assign(Date=data['Date'].dt.round('T'))['Date']
    times = []
    times.append(data['Date'][0])

    for i in data.index:
        time = data['Date'][i]
        if time != times[len(times)-1]:
            times.append(time)

    dic = {'time': [], f'{filename}': []}

    for hour in times:
        total = ''
        tweets = list(data[data['Date'] == hour]['Tweet'])
        for twit in tweets:
            total += str(twit)
        dic['time'].append(hour)
        dic[f'{filename}'].append(total)
    exec(f'data_{filename[1:]} = pd.DataFrame.from_dict(dic)')
    
    exec(f'data_{filename[1:]}.set_index("time", inplace=True)')
    exec(f'data_{filename[1:]}.sort_index(inplace=True)')
    
    exec(f'datum.append(data_{filename[1:]})')
df = pd.concat(datum, axis=1)
df.fillna('', inplace=True)

In [390]:
df['ALL'] = df['@TotalTrafficMIA'] + df['@TrafficAlertSFL'] + df['@nbc6'] + df['@CBSMiami'] + df['@wsvn'] +\
            df['@MiamiHerald'] +  df['@SunSentinel'] + df['@miaminewsnow'] + df['@BeachNewsAlerts'] + df['@WPLGLocal10'] +\
            df['@MiamiBeachPD'] + df['@miamibeachnews'] + df['@FLTurnpikeSFL']

In [391]:
df.head()

,@TotalTrafficMIA,@TrafficAlertSFL,@nbc6,@CBSMiami,@wsvn,@MiamiHerald,@SunSentinel,@WPLGLocal10,@miaminewsnow,@BeachNewsAlerts,@FLTurnpikeSFL,@MiamiBeachPD,@miamibeachnews,ALL
time,,,,,,,,,,,,,,
2019-01-01 00:02:00+00:00,,,,,,,Videos show staff of now-shuttered Arizona she...,,,,,,,Videos show staff of now-shuttered Arizona she...
2019-01-01 00:07:00+00:00,,,,,,,UM's Manny Diaz fires Hurricanes' offensive st...,,,,,,,UM's Manny Diaz fires Hurricanes' offensive st...
2019-01-01 00:35:00+00:00,,,,,,,Hurricanes linebacker Michael Pinckney returni...,,,,,,,Hurricanes linebacker Michael Pinckney returni...
2019-01-01 00:44:00+00:00,,,Before the party animals come out for tonight'...,,,,,,,,,,,Before the party animals come out for tonight'...
2019-01-01 00:47:00+00:00,Police activity; off-ramp 2 right lanes blocke...,,,,,,,,,,,,,Police activity; off-ramp 2 right lanes blocke...


In [239]:
(df.index[0] + pd.DateOffset(hours = 2) > df.index[j]) and (df.index[0] < df.index[j])

True

# X All

In [423]:
files = ['2019-07-30','2019-07-29','2019-07-28',
        '2019-07-27', '2019-07-26', '2019-07-25',
        '2019-07-24', '2019-07-23']

In [424]:
datum = []

for filename in tqdm_notebook(files):
    data = pd.read_csv(f'Datasets/timeloop_{filename}.csv')
    data.drop(columns=['User','User_ID','Geo'], inplace = True)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data.assign(Date=data['Date'].dt.round('T'))['Date']
    times = []
    times.append(data['Date'][0])

    for i in data.index:
        time = data['Date'][i]
        if time != times[len(times)-1]:
            times.append(time)

    dic = {'time': [], f'Tweet': []}

    for hour in times:
        total = ''
        tweets = list(data[data['Date'] == hour]['Tweet'])
        for twit in tweets:
            total += str(twit)
        dic['time'].append(hour)
        dic['Tweet'].append(total)
    exec(f'data_{filename[8:]} = pd.DataFrame.from_dict(dic)')
    
    exec(f'data_{filename[8:]}.set_index("time", inplace=True)')
    exec(f'data_{filename[8:]}.sort_index(inplace=True)')
    
    exec(f'datum.append(data_{filename[8:]})')
df = pd.concat(datum, axis=0)
df.fillna('', inplace=True)

In [425]:
df.head()

,Tweet
time,
2019-07-29 11:02:00+00:00,"Usted, izquierdista , guerrillero, no está en ..."
2019-07-29 11:03:00+00:00,The journey of a thousand miles begins with on...
2019-07-29 11:04:00+00:00,"""This Human Interaction Study Will Totally Cha..."
2019-07-29 11:05:00+00:00,Wow thats right Harvey took a Southerly track....
2019-07-29 11:06:00+00:00,Weather over next 8 hours at U of Miami Health...


# Format Y

In [429]:
streets = ['I-95', '95 Express', 'I-195', 'SR-826', 'SR-836', 'US-1',
           'US-41', 'I-75', 'US-441', 'SR-821', 'SR-9',
           'SR-997', 'SR-AIA']

In [430]:
data_y = pd.read_csv('Datasets/siteloop_@fl511_southeast.csv')
data_y.drop(columns=['User','User_ID','Geo', 'HashTag'], inplace = True)
data_y['Date'] = pd.to_datetime(data_y['Date'])

In [431]:
for street in tqdm_notebook(streets):
    road = pd.Series(index=data_y.index)
    for i in data_y.index:
        val = 0
        if (street in data_y['Tweet'][i]) and ('Cleared' in data_y['Tweet'][i]): 
            val = 1
        elif (street in data_y['Tweet'][i]) and ('lanes blocked' in data_y['Tweet'][i]): 
            val = 2
        elif (street in data_y['Tweet'][i]) and ('lane blocked' in data_y['Tweet'][i]): 
            val = 3
        elif (street in data_y['Tweet'][i]) and (('traffic' in data_y['Tweet'][i]) or ('closed' in data_y['Tweet'][i])):
            val = 4
        road[i] = val
    data_y[street] = road

In [432]:
data_y.head(20)

,Tweet,Date,I-95,95 Express,I-195,SR-826,SR-836,US-1,US-41,I-75,US-441,SR-821,SR-9,SR-997,SR-AIA
0,Cleared: Disabled vehicle in Miami-Dade on SR-...,2019-07-30 17:34:12+00:00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,New: Disabled vehicle in Miami-Dade on SR-826 ...,2019-07-30 17:29:06+00:00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cleared: Crash in Miami-Dade on Floridas Turnp...,2019-07-30 15:42:07+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cleared: Disabled vehicle in Miami-Dade on SR-...,2019-07-30 14:45:04+00:00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,New: Disabled vehicle in Miami-Dade on SR-826 ...,2019-07-30 14:36:03+00:00,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Cleared: Traffic congestion in Miami-Dade on F...,2019-07-30 13:56:04+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Cleared: Traffic congestion in Miami-Dade on F...,2019-07-30 13:45:08+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,Updated: Crash in Miami-Dade on Floridas Turnp...,2019-07-30 13:04:49+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Updated: Traffic congestion in Miami-Dade on F...,2019-07-30 12:58:09+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Cleared: Disabled vehicle in Miami-Dade on 95 ...,2019-07-30 12:17:11+00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Aggregate columns

In [452]:
df.head(1)

,Tweet
time,
2019-07-29 11:02:00+00:00,"Usted, izquierdista , guerrillero, no está en ..."


In [ ]:
july = data_y.iloc[0:2000]

In [456]:
agg_col = pd.Series(index=july.index)
for i in tqdm_notebook(july.index):
    agg_tweets = ''
    
    min_before = []
    
    for q in range(-30, 60):
        time = july['Date'][i].floor(freq='T') - pd.DateOffset(minutes=q)
        min_before.append(time)
        
    for j in min_before:
        try:
            agg_tweets += df['Tweet'][j]
        except:
            pass
    
    agg_col[i] = agg_tweets
july['Aggregate'] = agg_col

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [427]:
for i in tqdm_notebook(data_y.index):
    if data_y['Aggregate'][i] == '':
         data_y.drop(axis = 0, index = i, inplace=True)

In [414]:
for i in tqdm_notebook(data_y.index):
    if (data_y['I-95'][i] == 0) and (data_y['95 Express'][i] == 0) and (data_y['I-195'][i] == 0) and (data_y['SR-826'][i] == 0) and \
    (data_y['SR-836'][i] == 0) and (data_y['US-1'][i] == 0) and (data_y['US-41'][i] == 0) and (data_y['I-75'][i] == 0) and \
    (data_y['US-441'][i] == 0) and (data_y['SR-821'][i] == 0) and (data_y['SR-9'][i] == 0) and (data_y['SR-997'][i] == 0) and (data_y['SR-AIA'][i] == 0):
        data_y.drop(axis = 0, index = i, inplace=True)

In [460]:
X = july['Aggregate']
y = july['I-95']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [461]:
pipe = Pipeline([
    ('vec', TfidfVectorizer(token_pattern='[a-zA-z]+ | [A-Za-z]+\-*\d+\W(?:[sS]outh|[Nn]orth|East|West|[NSEW]{1,2}|[nswe]{1,2})*')),
    ('nb', MultinomialNB())
])
pipe_params = {
    'vec__stop_words': ['english'],
    'vec__max_features': [2000],
    'vec__min_df': [1],
    'vec__max_df': [.9],
    'vec__ngram_range': [(2,5)],
}

In [462]:
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print('cvs:', gs.best_score_)
print('train score:', gs.score(X_train, y_train))
print('test score:', gs.score(X_test, y_test))

cvs: 0.7466666666666667
train score: 0.7466666666666667
test score: 0.746


In [403]:
data_y['I-95'].value_counts(normalize=True)

0.0    0.797982
1.0    0.090174
2.0    0.056724
4.0    0.038867
3.0    0.016252
Name: I-95, dtype: float64